In this notebook, I will look at the various files generated by the fineSTRUCTURE run on chr8

In [2]:
%run ../scripts/notebook_settings.py
from sklearn.decomposition import PCA
from sklearn import preprocessing
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

In [3]:
chunkcounts_path = "/home/eriks/baboondiversity/people/eriks/baboon_first_analysis/steps/fs/chr8_no_gelada_linked.chunkcounts.out"
mcmcmatrix_path = "/home/eriks/baboondiversity/people/eriks/baboon_first_analysis/steps/fs/chr8_no_gelada/chr8_no_gelada_mcmcmatrix.txt"
metadata_path = "../data/Papio_metadata_with_clustering.txt"

In [4]:
chunkcounts = pd.read_csv(chunkcounts_path, sep=" ", header = 1, index_col=0)
mcmcmatrix = pd.read_csv(mcmcmatrix_path, sep=" ", index_col=0)
meta_data_samples = pd.read_csv(metadata_path, sep =" ")
meta_data_samples_Sci = meta_data_samples.copy()
for i, row in meta_data_samples_Sci.iterrows():
    if row.PGDP_ID[0] != "P":
        meta_data_samples_Sci.at[i, "PGDP_ID"] = "Sci_"+str(row.PGDP_ID)
meta_data_samples_Sci.index = meta_data_samples_Sci.PGDP_ID

Determining clusters by considering dendrogram, PCA, fineSTRUCTURE heatmap from chr8 and sampling location.

In [5]:
kindae = meta_data_samples_Sci.reindex(mcmcmatrix.index)[:27].C_origin
cyno_western = meta_data_samples_Sci.reindex(mcmcmatrix.index)[27:37].C_origin
ursinus = meta_data_samples_Sci.reindex(mcmcmatrix.index)[37:41].C_origin
cyno_ruaha = meta_data_samples_Sci.reindex(mcmcmatrix.index)[41:47].Origin
cyno_mikumi = meta_data_samples_Sci.reindex(mcmcmatrix.index)[47:84].Origin 
cyno_udzungwa = meta_data_samples_Sci.reindex(mcmcmatrix.index)[84:93].Origin #1 Mikumi is also in here PD_0687
anubis_manyara = meta_data_samples_Sci.reindex(mcmcmatrix.index)[93:112].Origin
anubis_na = meta_data_samples_Sci.reindex(mcmcmatrix.index)[112:122].Origin
anubis_tarangire = meta_data_samples_Sci.reindex(mcmcmatrix.index)[122:129].Origin
anubis_serengeti = meta_data_samples_Sci.reindex(mcmcmatrix.index)[129:143].Origin
anubis_gombe = meta_data_samples_Sci.reindex(mcmcmatrix.index)[143:160].Origin
anubis_ethiopia = meta_data_samples_Sci.reindex(mcmcmatrix.index)[160:185].Origin
papio_senegal = meta_data_samples_Sci.reindex(mcmcmatrix.index)[185:197].Origin
hamadryas_ethiopia = meta_data_samples_Sci.reindex(mcmcmatrix.index)[197:].Origin

In [16]:
anubis_ethiopia

PD_0730    Gog Woreda, Gambella region, Ethiopia
PD_0731    Gog Woreda, Gambella region, Ethiopia
PD_0723    Gog Woreda, Gambella region, Ethiopia
PD_0735    Gog Woreda, Gambella region, Ethiopia
PD_0722    Gog Woreda, Gambella region, Ethiopia
PD_0721    Gog Woreda, Gambella region, Ethiopia
PD_0725    Gog Woreda, Gambella region, Ethiopia
PD_0720    Gog Woreda, Gambella region, Ethiopia
PD_0726    Gog Woreda, Gambella region, Ethiopia
PD_0727    Gog Woreda, Gambella region, Ethiopia
PD_0732    Gog Woreda, Gambella region, Ethiopia
PD_0742    Gog Woreda, Gambella region, Ethiopia
PD_0729    Gog Woreda, Gambella region, Ethiopia
PD_0728    Gog Woreda, Gambella region, Ethiopia
PD_0741    Gog Woreda, Gambella region, Ethiopia
PD_0743    Gog Woreda, Gambella region, Ethiopia
PD_0737    Gog Woreda, Gambella region, Ethiopia
PD_0724    Gog Woreda, Gambella region, Ethiopia
PD_0733    Gog Woreda, Gambella region, Ethiopia
PD_0738    Gog Woreda, Gambella region, Ethiopia
PD_0739    Gog Wored

In [ ]:
pop_df = pd.DataFrame()
pop_df["ID"] = meta_data_samples_Sci.reindex(mcmcmatrix.index).PGDP_ID
pop_df["population"] = "unknown"
pop_df["inclusion"] = "unknown"
pop_df.iloc[0:27,1], pop_df.iloc[0:27,2] = "kindae", 1
pop_df.iloc[27:37,1], pop_df.iloc[27:37,2] = "cyno_western", 1
pop_df.iloc[37:41,1], pop_df.iloc[37:41,2] = "ursinus", 1
pop_df.iloc[41:78,1], pop_df.iloc[41:78,2] = "cyno_mikumi", 1
pop_df.iloc[78:93,1], pop_df.iloc[78:93,2] = "cyno_RS", 1
pop_df.iloc[93:110,1], pop_df.iloc[93:110,2] = "anubis_gombe", 1
pop_df.iloc[110:112,1], pop_df.iloc[110:112,2] = "anubis_kenya", 0
pop_df.iloc[112:126,1], pop_df.iloc[112:126,2] = "anubis_serengeti", 1
pop_df.iloc[126:162,1], pop_df.iloc[126:162,2] = "anubis_MGAT", 1
pop_df.iloc[162:187,1], pop_df.iloc[162:187,2] = "anubis_ethiopia", 1
pop_df.iloc[187:189,1], pop_df.iloc[187:189,2] = "gelada", 0
pop_df.iloc[189:201,1], pop_df.iloc[189:201,2] = "papio", 1
pop_df.iloc[201:,1], pop_df.iloc[201:,2] = "hamadryas", 1

In [ ]:
zero_PGDP_ID = []
for population in pop_df.population.unique():
    sub_df = pop_df.loc[pop_df.population==population]
    zero_PGDP_ID.extend(sub_df["ID"].sample(n=max(len(sub_df), 4)-4))
pop_df_subset = pop_df.copy()
for ID in zero_PGDP_ID:
    pop_df_subset.loc[pop_df_subset.ID==ID, "inclusion"] = 0

In [ ]:
pop_df_reorder = pop_df.reindex(meta_data_samples_Sci.index)
pop_df_reorder.to_csv("/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/idfile_fs_cluster.ids",
              sep=" ", header=False, index=False)

In [ ]:
pop_df_subset_reorder = pop_df_subset.reindex(meta_data_samples_Sci.index)
pop_df_subset_reorder.to_csv("/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/idfile_subset.ids",
              sep=" ", header=False, index=False)

Subset for only the females in the chrX vcf. Alan has followed my gendering, so that is nice.

In [ ]:
vcf_x = "/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/chrX/chrX.phased.rehead.vcf.gz"
callset = allel.read_vcf(vcf_x)
pop_df_females = pop_df.reindex(callset["samples"])

In [ ]:
pop_df_females.to_csv("/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/chrX_females.ids",
              sep=" ", header=False, index=False)

In [ ]:
pop_df_females

Idfile containing only the 8 clusters originally determined

In [ ]:
pop_df = pd.DataFrame()
pop_df["ID"] = meta_data_samples_Sci.reindex(mcmcmatrix.index).PGDP_ID
pop_df["population"] = "unknown"
pop_df["inclusion"] = "unknown"
pop_df.iloc[0:27,1], pop_df.iloc[0:27,2] = "kindae", 1
pop_df.iloc[27:37,1], pop_df.iloc[27:37,2] = "cyno_western", 1
pop_df.iloc[37:41,1], pop_df.iloc[37:41,2] = "ursinus", 1
pop_df.iloc[41:93,1], pop_df.iloc[41:93,2] = "cyno_central", 1
pop_df.iloc[93:110,1], pop_df.iloc[93:110,2] = "anubis_tanzania", 1
pop_df.iloc[110:112,1], pop_df.iloc[110:112,2] = "anubis_kenya", 0
pop_df.iloc[112:126,1], pop_df.iloc[112:126,2] = "anubis_tanzania", 1
pop_df.iloc[126:162,1], pop_df.iloc[126:162,2] = "anubis_tanzania", 1
pop_df.iloc[162:187,1], pop_df.iloc[162:187,2] = "anubis_ethiopia", 1
pop_df.iloc[187:189,1], pop_df.iloc[187:189,2] = "gelada", 0
pop_df.iloc[189:201,1], pop_df.iloc[189:201,2] = "papio", 1
pop_df.iloc[201:,1], pop_df.iloc[201:,2] = "hamadryas", 1

In [ ]:
pop_df_reorder = pop_df.reindex(meta_data_samples_Sci.index)
pop_df_reorder.to_csv("/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/idfile_8_cluster.ids",
              sep=" ", header=False, index=False)

In [ ]:
zero_PGDP_ID = []
for population in pop_df.population.unique():
    sub_df = pop_df.loc[pop_df.population==population]
    zero_PGDP_ID.extend(sub_df["ID"].sample(n=max(len(sub_df), 10)-10))
pop_df_subset = pop_df.copy()
for ID in zero_PGDP_ID:
    pop_df_subset.loc[pop_df_subset.ID==ID, "inclusion"] = 0
pop_df_subset_reorder = pop_df_subset.reindex(meta_data_samples_Sci.index)
pop_df_subset_reorder.to_csv("/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/idfile_8_cluster_subset.ids",
              sep=" ", header=False, index=False)

Repeat for 8 cluster, females

In [ ]:
vcf_x = "/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/chrX/chrX.phased.rehead.vcf.gz"
callset = allel.read_vcf(vcf_x)
pop_df_females = pop_df.reindex(callset["samples"])
pop_df_females.to_csv("/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/chrX_8_cluster_females.ids",
              sep=" ", header=False, index=False)

pop_df_females

In [ ]:
f = open("/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/bcf_inclusion.txt", "w")
included_inds = (pop_df_females.loc[pop_df_females.inclusion==1].ID.values)
for ind in included_inds:
    f.write(ind+"\n")

In [ ]:
pop_df_females.loc[pop_df_females.inclusion==1].to_csv("/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/included_females.ids",
              sep=" ", header=False, index=False)

In [ ]:
",".join(pop_df_females.loc[pop_df_females.inclusion==1].ID.values)+"\n"

In [ ]:
chunkcounts.loc[chunkcounts.max() > 4000]

PCA

In [ ]:
in_df = meta_data_samples_Sci.loc[meta_data_samples_Sci["PGDP_ID"].isin(chunkcounts.index)]
sorted_origin = in_df.sort_values(by=["C_origin"])
sorted_chunks = chunkcounts.reindex(sorted_origin["PGDP_ID"])
sorted_chunks = sorted_chunks.reindex(columns=sorted_origin["PGDP_ID"])
merged_chunk_meta = pd.merge(sorted_chunks, in_df, on="PGDP_ID")

In [ ]:
pca = PCA(n_components=10)
X_normalized = preprocessing.normalize(sorted_chunks, norm='l2')
projection = pca.fit_transform(X_normalized)

print(pca.explained_variance_)

In [ ]:
pcx, pcy = 1, 0
g = sns.scatterplot(x=projection[:,pcx], y=projection[:,pcy], hue=merged_chunk_meta["C_origin"])
g.legend(bbox_to_anchor=(1.02, 1.02))
g.set(xlabel="pc{}".format(pcx+1), ylabel="pc{}".format(pcy+1))

I can either generate clusters based on chunkcounts, or try to extract the tree generated by fineSTRUCTURE and analyze that. I will first try to generate it based on chunkcounts/PCA results

In [ ]:
chunkpercentage = chunkcounts.div(chunkcounts.sum(axis=1), axis=0)

For each individual, I will make a list of individuals over a certain threshold of relatedness.